In [ ]:
from packaging.version import Version, parse
from typing import List, Tuple

In [ ]:
def parse_pragma(pragma: str) -> Tuple[str, str]:
    parts = pragma.split()
    version_spec = parts[2].replace(";", "")
    if version_spec[0] in ['^', '>', '<', '=', '>=', '<=']:
        operator = version_spec[0]
        ver = version_spec[1:] if operator != '=' else version_spec
    else:
        operator = '='
        ver = version_spec
    if version_spec.startswith('>=') or version_spec.startswith('<='):
        operator = version_spec[:2]
        ver = version_spec[2:]
    return operator, ver

def next_major(ver: Version) -> Version:
    return parse(f"{ver.major + 1}.0.0")

def satisfies_constraint(version: Version, operator: str, constraint: Version) -> bool:
    if operator == '^':
        return constraint <= version < next_major(constraint)
    elif operator == '>':
        return version > constraint
    elif operator == '<':
        return version < constraint
    elif operator == '>=':
        return version >= constraint
    elif operator == '<=':
        return version <= constraint
    elif operator == '=':
        return version == constraint

def find_compatible_version(versions: List[str]) -> str:
    try:
        constraints = [parse_pragma(v) for v in versions]
        possible_versions = [parse(f"0.{i}.0") for i in range(1, 13)]

        for possible_version in possible_versions:
            if all(satisfies_constraint(possible_version, op, parse(ver)) for op, ver in constraints):
                return f"{possible_version}"
    except:
        pass
    return None


In [105]:
from packaging.version import parse, Version
from typing import List, Tuple, Set

def parse_pragma(pragma: str) -> Tuple[str, str]:
    parts = pragma.split()
    version_spec = parts[2].replace(";", "")
    if version_spec.startswith(('>=', '<=', '^')):
        operator = version_spec[:2]
        ver = version_spec[2:]
    elif version_spec[0] in ['>', '<', '=']:
        operator = version_spec[0]
        ver = version_spec[1:]
    else:
        operator = '='
        ver = version_spec
    return operator, ver

def deduplicate_constraints(constraints: List[Tuple[str, str]]) -> List[Tuple[str, Version]]:
    """Remove duplicate constraints and parse versions."""
    unique_constraints: Set[Tuple[str, str]] = set(constraints)
    return [(op, parse(ver)) for op, ver in unique_constraints]

def satisfies_all_constraints(version: Version, constraints: List[Tuple[str, Version]]) -> bool:
    for operator, constraint in constraints:
        if operator == '^':
            if not (constraint <= version < next_major(constraint)):
                return False
        elif operator == '>':
            if not version > constraint:
                return False
        elif operator == '<':
            if not version < constraint:
                return False
        elif operator == '>=':
            if not version >= constraint:
                return False
        elif operator == '<=':
            if not version <= constraint:
                return False
        elif operator == '=':
            if not version == constraint:
                return False
    return True

def next_major(ver: Version) -> Version:
    return parse(f"{ver.major + 1}.0.0")

def find_compatible_version(pragmas: List[str]) -> str:
    constraints = deduplicate_constraints([parse_pragma(pragma) for pragma in pragmas])
    for major in range(0, 13):  # Assuming Solidity versions up to 0.12.x are relevant
        for minor in range(0, 10):  # Minor version loop
            version = parse(f"{major}.{minor}.0")
            if satisfies_all_constraints(version, constraints):
                return f"Compatible compiler version found: {version}"
    return "No compatible compiler version found."

# Example usage
pragmas = [
    "pragma solidity ^0.6.0;",
    "pragma solidity ^0.6.0;",
    "pragma solidity ^0.6.0;",
    "pragma solidity ^0.6.2;",
    "pragma solidity ^0.6.0;",
    "pragma solidity ^0.6.0;"
]
print(find_compatible_version(pragmas))


InvalidVersion: Invalid version: '.6.0'

In [102]:

# Example usage with the corrected function
pragmas = [
    "pragma solidity ^0.4.0;",
    "pragma solidity >0.7.0;",
    "pragma solidity >=0.7.2 <0.8.0;",
]
print(find_compatible_version(pragmas))

Compatible compiler version found: 0.8.0


In [103]:
# Example usage
pragmas = [
    "pragma solidity ^0.8.0",
    "pragma solidity >=0.7.0 <0.9.0",
]
print(find_compatible_version(pragmas))


Compatible compiler version found: 0.8.0


In [104]:
pragmas = [
    "pragma solidity ^0.8.0;",
    "pragma solidity >0.4.0;",
    "pragma solidity >=0.7.2 <0.8.0;",
]
print(find_compatible_version(pragmas))


Compatible compiler version found: 0.8.0


In [ ]:
import os
import re

# Initialize an empty dictionary to hold the file paths and pragma declarations
pragma_dict = {}

# Define the root folder path (assume the script is running in the root folder)
root_folder = '.'  # Current directory as root

# Regular expression to match pragma declarations
pragma_pattern = re.compile(r'^\s*pragma\s+[^;]+;')

# Function to add pragma declarations from a file to the pragma_dict
def add_pragma_declarations(file_path):
    with open(file_path, 'r') as file:
        content = file.readlines()
        pragmas = [line.strip() for line in content if pragma_pattern.match(line)]
        if pragmas:
            # Use the relative path from the root folder as the key
            relative_path = os.path.relpath(file_path, start=root_folder)
            pragma_dict[relative_path] = pragmas

# Traverse the folder structure
for root, dirs, files in os.walk(root_folder):
    # Limit the depth to 3 levels including the root folder
    depth = root.count(os.sep) - root_folder.count(os.sep)
    if depth <= 10:  # Adjust depth according to your requirement
        for file in files:
            if file.endswith('.sol'):
                file_path = os.path.join(root, file)
                add_pragma_declarations(file_path)
    else:
        # Skip deeper directories
        dirs[:] = []  # Remove dirs to not go deeper

# Now, pragma_dict contains the desired information
print(pragma_dict)


In [ ]:
compatible_pragmas = dict()
count_nones = 0
for file, pragmas in pragma_dict.items():
    result = find_compatible_version(pragmas)
    if result is None:
        count_nones += 1
    compatible_pragmas[f"{file}"] = result

In [ ]:
len(compatible_pragmas)

In [ ]:
count_nones

In [ ]:
compatible_pragmas

In [ ]:
pragma_dict['compilable_DAppSCAN/Chainsulting-GSPI Club-project2/openzeppelin-contracts-3.2.0/contracts/payment/escrow/ConditionalEscrow.sol']

In [ ]:
[
    '0.4.11',
    '0.4.18',
    '0.4.'
]